In [2]:
#import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#create a set of random latitude and longitude combinations
lats = np.random.uniform(low = -90, high = 90, size = 1500)
lngs = np.random.uniform(low = -180, high = 180, size = 1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
coordinates = list(lat_lngs)

In [5]:
from citipy import citipy

In [6]:
# create a list for holding the cities
cities = []
#identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name

    #if city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# print the city count to confirm suficiient count
len(cities)

608